In [ ]:
from pyturbo.systems.gas_generator import GasGenerator

In [ ]:
from cosapp.drivers import NonLinearSolver
from cosapp.utils import LogLevel, set_log
from cosapp.recorders import DataFrameRecorder

set_log()

In [ ]:
core = GasGenerator("core")

core.kp.inlet_tip[0] = 0.25
core.kp.exit_tip[0] = 0.25
core.fuel_W = 0.11
core.turbine.aero.Ncdes = 15.
core.compressor.geom.blade_hub_to_tip_ratio = 0.8
core.turbine.sh_out.power = 30e6
core.turbine.geom.blade_height_ratio = 0.1

core.fl_in.Tt = 409.
core.fl_in.Pt = 2.67e5

core.compressor.sh_in.power = core.turbine.sh_out.power

core.compressor.geom.blade_hub_to_tip_ratio = 0.7

run = core.add_driver(NonLinearSolver('run', factor=0.8, history=True))

run.add_equation("turbine.aero.Ncqdes == 100.").add_unknown("compressor.fl_in.W", max_rel_step=0.8)
run.add_equation("compressor.aero.phi == 0.45").add_unknown("compressor.aero.phiP")
run.add_equation("compressor.aero.pr == 11.")
run.add_equation("combustor.aero.Tcomb == 1650.").add_unknown("fuel_W")
run.add_equation("compressor.aero.utip == 450.").add_unknown("turbine.aero.Ncdes")
run.add_unknown("kp.inlet_tip[0]", lower_bound=1e-5, max_rel_step=0.8).add_target("turbine.sh_out.power")
run.add_unknown("kp.exit_tip[0]", lower_bound=1e-5, max_rel_step=0.8).add_equation("turbine.aero.psi == 1.1")
run.add_unknown("compressor.aero.xnd").add_equation("compressor.aero.pcnr == 98.")
run.add_unknown("turbine.geom.blade_height_ratio", lower_bound=0., upper_bound=1.)

rec = run.runner.add_recorder(DataFrameRecorder(includes='*'))
                                                            
core.run_drivers()

In [ ]:
run.problem

In [ ]:
run = core.add_driver(NonLinearSolver('run', factor=0.95, history=True))
run.add_unknown("fuel_W").add_equation("compressor.aero.pcnr == 95.")
run.add_unknown("compressor.fl_in.W").add_target("turbine.aero.fl_out.Wc")

core.run_drivers()

In [ ]:
core.turbine.aero.dhqt, core.turbine.aero.Ncqdes

In [ ]:
core.jupyter_view(options={
    "compressor": dict(opacity=0.8),
    "combustor": dict(face_color="red", opacity=0.8),
    "turbine": dict(opacity=0.8),
})